POC Project: Google FLAN-T5-Large + HuggingFaceEmbedding + LlamaIndexing

autratec 2023

My overall impression of Google Flan is that it feels like talking to someone at a low primary school level, while GPT4ALL LLM is comparable to a primary 5 or 6 student. ChatGPT, on the other hand, is more like a secondary student.

The good news is that running Google Flan won't require too many resources from Google Coblab, although it will still need to be connected to a GPU. However, it cannot be used for commercial or real-life environments. 

In [ ]:
! pip install -q langchain transformers sentence_transformers llama-index

In [ ]:
import torch

In [ ]:
from llama_index import SimpleDirectoryReader, LangchainEmbedding, GPTSimpleVectorIndex, PromptHelper, LLMPredictor, ServiceContext
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.llms.base import LLM
from transformers import pipeline

In [ ]:
embed_model = LangchainEmbedding(HuggingFaceEmbeddings())

In [ ]:
max_input_size = 512
num_output = 200
max_chunk_overlap = 20
chunk_size_limit = 200

In [ ]:
class customLLM(LLM):
    model_name = "google/flan-t5-large"
    pipeline = pipeline("text2text-generation", model=model_name, device=0, model_kwargs={"torch_dtype":torch.bfloat16})
#    pipeline = pipeline("text2text-generation", model=model_name, device='cpu', model_kwargs={"torch_dtype":torch.bfloat16})
    def _call(self, prompt, stop=None):
        return self.pipeline(prompt, max_length=9999)[0]["generated_text"]
    def _identifying_params(self):
        return {"name_of_model": self.model_name}
    def _llm_type(self):
        return "custom"
llm_predictor = LLMPredictor(llm=customLLM())

Simple test to ensure llm is working

In [ ]:
llm=customLLM()
print(llm._call("Hi! How is everything going today ?"))

In [ ]:
prompt_helper = PromptHelper(max_input_size, num_output,max_chunk_overlap,chunk_size_limit=chunk_size_limit)
service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor, embed_model=embed_model, prompt_helper=prompt_helper, chunk_size_limit = chunk_size_limit) 

Pls create a folder under content, call "data". Load your txt file for indexing. you should find a file called index.json after completeting the indexing(embedding)

In [ ]:
documents = SimpleDirectoryReader('./data').load_data()
index = GPTSimpleVectorIndex.from_documents(documents,service_context=service_context)
index.save_to_disk('index.json')

In [ ]:
query_text = "How to manage the project chage ?"
response = index.query(query_text,response_mode="compact",service_context=service_context, similarity_top_k=3)
print(response)

Change management is the process whereby changes to work are formally introduced and approved
